```{shiny}
#| standalone: true
#| viewerHeight: 600
library(shiny)
library(bslib)
library(tidyverse)
library(ggplot2)

# Define UI for app that draws a histogram ----
ui <- page_sidebar(
  sidebar = sidebar(open = "open",
                    # selecting countries from a list of checkboxes
                    selectizeInput("country", "Select Countries", choices = NULL, multiple = TRUE),
                    # make a slider input for years
                    sliderInput("year", "Select Year", min = 2000, max = 2020, value = c(2000, 2020)),
  ),
  plotOutput("plot", width=1100)
)

# Define the server logic
server <- function(input, output, session) {
  
  # Load and preprocess the data
  data <- reactive({
      read.csv("data/API_NY.GDP.PCAP.PP.CD_DS2_en_csv_v2_204/API_NY.GDP.PCAP.PP.CD_DS2_en_csv_v2_204.csv",
             sep = ";", header = TRUE, skip = 4) %>%
      rename_with(~ gsub("X", "", .x)) %>%
      select(-c(3:5)) %>%
      pivot_longer(cols = -c(1:2), names_to = "Year", values_to = "Value") %>%
      mutate(Year = as.numeric(Year))
  })
  
  # Update the country choices
  observe({
    updateSelectizeInput(session, "country", choices = unique(
      data()$Country.Name))
  })
    
  # Filter the data
  filtered_data <- reactive({
    req(input$country, input$year)
    data() %>%
      filter(`Country.Name` %in% input$country,
             Year >= input$year[1],
             Year <= input$year[2])
  })
  
  # Create the plot
  output$plot <- renderPlot({
    ggplot(filtered_data(), aes(x = Year, y = Value, color = `Country.Name`)) +
      geom_line(na.rm = TRUE) + # Add na.rm = TRUE to remove NA values
      labs(title = "GDP PPP Over Time",
           x = "Year",
           y = "GDP PPP")
  })

}

# Run the application
shinyApp(ui = ui, server = server)

```